In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold


In [3]:
#Load in the data
X = pd.read_csv("../Data/trainX.csv")
y = pd.read_csv("../Data/trainY.csv")

In [4]:
# create a 75/25 stratified split of the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7, test_size=0.25)

In [5]:
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (2249, 44) (2249, 1)
Test set: (750, 44) (750, 1)


# Modeling

## Random Forest

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [7]:
# Scaling features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
# Make Random Forest model exploring n_estimators
regressor = RandomForestRegressor(n_estimators=200, random_state=0)
regressor.fit(X_train, y_train.values.ravel())
y_pred = regressor.predict(X_test)

# Mean error calculations
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

# Calculate absolute errors
errors = abs(y_pred - y_test.values)

# Calculate mean absolute percentage error
mape = 100 * (errors / y_test.values)

# Calculate and display percent accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%')

Mean Absolute Error: 1.4095207476586256
Mean Squared Error: 4.619481098171461
Root Mean Squared Error: 2.1492978151413684
Accuracy: 73.2 %
